In [1]:
from os import scandir, getcwd
import pandas as pd
import time
import numpy as np
import glob
import unidecode

def ls(ruta = getcwd()):
    return [arch.name for arch in scandir(ruta) if arch.is_file()]

In [ ]:
SOLICITUDES = []
for i in ls():
    if "SOLICITUDES" in i: #and "REGISTRO" not in i:
        SOLICITUDES.append(i)
    if "solicitudes" in i: #and "registro" not in i:
        SOLICITUDES.append(i)
        
SOLICITUDES

In [ ]:
salida = []
derechos_aux = []
for i in SOLICITUDES:
    index = 0
    #flag = True
    while True:
        columna1 = pd.read_excel(i, skiprows=index).columns[0]
        if columna1 == "N°" or columna1 == "Número" or columna1 == "Numero" or columna1 == "Numero ":
            derechos_aux.append(i)
            salida.append(pd.read_excel(i, skiprows=index).copy())
            break
        if(index > 10):
            print(i)
            break
        index += 1

In [ ]:
for i in range(len(salida)):
    if(len(salida[i].columns) != 39):
        print(i)
        print(SOLICITUDES[i])

In [ ]:
for i in range(len(salida)):
    print(i)
    print(SOLICITUDES[i])
    print(len(salida[i].columns))
    print("*********************************************")

In [ ]:
columnas = []
for i in salida[0].columns:
    print(i.replace("\n",""))
    columnas.append(i.replace("\n",""))

In [ ]:
for i in salida[4].columns:
    if i not in columnas:
        print(i)

In [ ]:
for i in range(len(salida[0].columns)):
    try:
        print(salida[0].columns[i])
        print("-")
        print(salida[4].columns[i])
    except:
        pass
        #print(salida[0].columns[i])
    print("***************************************")

In [ ]:
def verificarColumnas(data, referencia):
    ruta = "Referencia/"
    df = data
    df = df[df.columns.dropna()]
    df = df.loc[:,~df.columns.duplicated()]
    
    ref = pd.read_excel(ruta + referencia, skiprows=4)
    
    existe = False
    columna = 0    
    #Eliminar Columnas que sobran    
    for col in df.columns:        
        for colRef in ref.columns:
            
            colCom = col
            colRefCom = colRef
            
            if "\n" in colRef.lower():
                colRefCom = colRef.replace("\n","")
            if "\n" in col.lower():
                colCom = col.replace("\n","")
            if unidecode.unidecode(colCom.lower()) == unidecode.unidecode(colRefCom.lower()):
                #print("existe la columna" + col)
                existe = True
            if "número" in col.lower() or "numero" in col.lower():
                existe = True
            if "region" in col.lower() or "región" in col.lower():
                existe = True
            if "codigo de expediente" in col.lower() or "código de expediente" in col.lower():
                existe = True
                
        if existe == False:
            
            del df[col]
            #df.to_csv(csvPorVerificar, index = False)
            print("se Eliminó la columna " + col)
            
        existe = False   
    #Agregar columnas faltantes    
    for colRef in ref.columns:        
        for col in df.columns:
            if col.lower() == colRef.lower():
                #print("existe la columna" + col)
                existe = True
        if "\n" in colRef.lower():
            colRefCom = colRef.replace("\n","")
        if colRef == "Región":
            existe = True
        if colRef == "Region":
            existe = True
        if colRefCom.lower() == "código de expediente":
            existe = True
        if colRefCom.lower() == "codigo de expediente":
            existe = True
            
        if existe == False:
            
            df.insert(columna,colRef,'')
            #df.to_csv(csvPorVerificar, index = False)
            print("se agregó la columna " + colRef)
        columna+=1
        existe = False
    '''    
    if "Region" in str(df.iloc[0]):
       
        region = df["Region"]
        del df["Región"]
        df.insert(3, "Región", region)
        del df["Region"]'''
    
    df.rename(columns = {"Region":"Región", "Numero":"Número", "Codigo de Expediente":"Código de Expediente", "Codigo de \nExpediente":"Código de Expediente", "Código de \nExpediente":"Código de Expediente" }, inplace = True)
    #df.rename(columns = {"Numero":"Número"}, inplace = True)
    #print(df)
    
    if "Número" in str(df.iloc[0]):
        del df["N°"]
        df.insert(0, "N°", df["Número"])
        del df["Número"]
    #print(df)
    if "Código de Expediente" in str(df.iloc[0]):
        Expediente = df["Código de Expediente"]
        del df["Código de Expediente"]
        df.insert(1, "Código de Expediente", Expediente)
    
    #print(df)
    
    return df

In [ ]:
for i in range(len(salida)):
    print(i)
    print(SOLICITUDES[i])
    tabla = pd.read_excel(SOLICITUDES[i])
    
    #print(tabla)
    contador = 0
   
    for j in tabla.iloc[:,0]:
        
        if "\n" in str(j):
            j = str(j).replace("\n","")
        #print(j)
        if "N°" in str(j) or "Número" in str(j) or "Numero" in str(j):
            
            new_header = tabla.iloc[contador]
            tabla = tabla[contador+1:]
            tabla.columns = new_header
           
            break
        contador = contador + 1
            
        
    
    df = verificarColumnas(tabla, "SOLICITUDES_INGRESADAS_EN_OCTUBRE_DE_2018.xls")
    
    df.to_excel("RESULTADO/"+SOLICITUDES[i], index = False)
    print("########################### DIVISION ######################")

In [20]:
file1 = "Datos Solicitudes Parciales/*.xls"
for item in file1:
    files = glob.glob(file1)
    df = pd.DataFrame()
    for file in files:
        #df1 = 
        
            #df1[i] = df1[i].replace("a", "á").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u")
        
        df = df.append(pd.read_excel(file), ignore_index=True)
    
for i in df.columns:

    for j in df.columns:

        b = i
        if i.lower().replace("á", "a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u").replace("\n","") == j.lower().replace("á", "a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u").replace("\n",""):
            
            b = b.replace("\n","")
            df.rename(columns = {str(j) : str(b)}, inplace = True)
    if str(i) == "Desnivel":
        print("Desnivel FINAL")
        break
                
df = df.groupby(level=0, axis=1, sort = False).first()

df.drop(df.columns[34:], axis=1, inplace=True)
#nombre = item.replace("REGIONAL/","")

df.to_excel("Datos Solicitudes Consolidado.xlsx", index = False)

Desnivel FINAL


In [ ]:
dfs = pd.read_excel("REGISTRO_CBR_DE_DERECHOS_NC_RECIBIDOS_EN_EL_MES_DE_MAYO_2012.xls")
dfs